In [95]:
UNIV_COLUMNS = ['대학코드', '국가', '대학명', '파견구분', '협정형태', 'NUM_VISITS', "BERT_SUM", "RNN_SUM", 'href']
ABSTRACT_REVIEWS_COLUMNS = ['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']
SPECIFIC_COLUMNS = ["대학코드", "대학명", "국가", "파견구분", "협정형태", "학과", "제목", "과정", "년도", "college_link", "review_link",  "gen_info","env_info","food_info","study_info","office_info" "facil_info","mhct_info","help_info","etc_info", "BERT_SCORE", "RNN_SCORE"]

In [96]:
# import all modules
import os
from os import fdopen, remove
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

import pandas as pd
import numpy as np

## wrangle abstract review dataset

In [97]:
# define path for sentiment labeled dataset
# abstract: 짧은 후기 제목만 있는 텍스트 데이터셋
# specific: 문단으로 구성된 텍스트 데이터셋

abstract_sentiment = "./data_sentiment/abstract"
specific_sentiment = "./data_sentiment/specific"

In [98]:
# fetch all yonsei exchange review text datasets for each foreign universities
abstract_yonsei_reviews = glob.glob(f"{abstract_sentiment}/*.csv")
abstract_yonsei_reviews.sort()
abstract_yonsei_reviews[:5]

['./data_sentiment/abstract/730     _review_abstract_sentiment.csv',
 './data_sentiment/abstract/AR000004_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000001_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000002_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000003_review_abstract_sentiment.csv']

In [99]:
# get sample dataset
sample_file = abstract_yonsei_reviews[70]

# extract file name
file_name = sample_file.split("/")[-1]
file_name_without_ext = file_name.split(".")[0]
print(file_name_without_ext)

# extract university code
univ_code_sample = file_name_without_ext.split("_")[0]
print(univ_code_sample)

# read sample dataset
df_abstract = pd.read_csv(sample_file, encoding="utf-8")
df_abstract.sample(5)

CN000022_review_abstract_sentiment
CN000022


,Unnamed: 0,No,제목,학과,과정,년도,href,BERT_SCORE,RNN_SCORE
3,3,15,북경대학교 교환생활,중어중문학과,학부,2018,/partner/expReport.asp?id=15513&page=1&bgbn=R,0,0.602057
0,0,18,PKU Exchange Program,언더우드국제대학,학부,2019,/partner/expReport.asp?id=16451&page=1&bgbn=R,0,0.515649
10,10,8,북경대 교환생활,사학과,학부,2016,/partner/expReport.asp?id=13623&page=2&bgbn=R,0,0.484792
7,7,11,가을학기 북경대학교 교환학생 파견,정치외교학과,학부,2017,/partner/expReport.asp?id=14422&page=1&bgbn=R,1,0.800448
8,8,10,북경대학교 경험보고서,아시아학부,학부,2017,/partner/expReport.asp?id=14387&page=1&bgbn=R,1,0.775683


In [103]:
df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code_sample
_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code_sample
df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,CN000022,PKU Exchange Program,언더우드국제대학,학부,2019,0,0.515649,/partner/expReport.asp?id=16451&page=1&bgbn=R
1,CN000022,베이징대 교환생활,중어중문학과,학부,2019,0,0.484792,/partner/expReport.asp?id=16194&page=1&bgbn=R
2,CN000022,북경대학교 교환학생,정치외교학과,학부,2019,0,0.664221,/partner/expReport.asp?id=15979&page=1&bgbn=R
3,CN000022,북경대학교 교환생활,중어중문학과,학부,2018,0,0.602057,/partner/expReport.asp?id=15513&page=1&bgbn=R
4,CN000022,북경대학교에서의 한 학기 교환생활,경영학과,학부,2018,1,0.697339,/partner/expReport.asp?id=15209&page=1&bgbn=R


In [104]:
df_abstract_review.columns.to_list()

['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']

In [105]:
def make_abstract_review_df(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    df_abstract = pd.read_csv(file_path, encoding="utf-8")
    df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = UNIV_CODE
    _abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = UNIV_CODE
    df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    return df_abstract_review

In [106]:
df1 = make_abstract_review_df("DK000003")
df1.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,DK000003,1년의 휘게!,건축학과,학부,2018-2019,1,0.414879,/partner/expReport.asp?id=16597&page=1&bgbn=R
1,DK000003,1년의 Hygge,국제학과,학부,2019,1,0.523097,/partner/expReport.asp?id=16079&page=1&bgbn=R
2,DK000003,코펜하겐 대학교에서의 한 학기! 최고!,경제학과,학부,2019,1,0.931257,/partner/expReport.asp?id=16028&page=1&bgbn=R
3,DK000003,인생에서 가장 행복했던 1년!,심리학과,학부,2019,1,0.967065,/partner/expReport.asp?id=15938&page=1&bgbn=R
4,DK000003,코펜하겐대학교 교환 가세요ㅠㅠㅠㅠ 고민하지 마세요!,언론홍보영상학부,학부,2019,0,0.651389,/partner/expReport.asp?id=15829&page=1&bgbn=R


In [107]:
df2 = make_abstract_review_df("CN000016")
df2.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,CN000016,"홍콩대,",UIC,학부,2018,0,0.523097,/partner/expReport.asp?id=16495&page=1&bgbn=R
1,CN000016,홍콩대 교환학생,언더우드국제학부,학부,2019,0,0.523097,/partner/expReport.asp?id=16326&page=1&bgbn=R
2,CN000016,홍콩대학교에서의 한 학기,응용통계학과,학부,2019,1,0.744596,/partner/expReport.asp?id=15804&page=1&bgbn=R
3,CN000016,홍콩대학교 교환 후기,언더우드학과,학부,2019,0,0.640901,/partner/expReport.asp?id=15751&page=1&bgbn=R
4,CN000016,홍콩대학교에서의 한학기,불어불문학과,학부,2019,1,0.744596,/partner/expReport.asp?id=15710&page=1&bgbn=R


In [108]:
df_concat = pd.concat([df1,df2])
df_concat.sample(5)

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
36,DK000003,신 맛 코펜하겐,언론홍보영상/사회학,학부,2009,1,0.497112,/partner/expReport.asp?id=5327&page=4&bgbn=R
0,CN000016,"홍콩대,",UIC,학부,2018,0,0.523097,/partner/expReport.asp?id=16495&page=1&bgbn=R
27,CN000016,홍콩에서의 잊지못할 경험!,국어국문학과,학부,2015,1,0.959675,/partner/expReport.asp?id=12344&page=3&bgbn=R
30,DK000003,고생한만큼 즐거웠던 코펜하겐!,신문방송학과,학부,2011,1,0.920942,/partner/expReport.asp?id=6650&page=4&bgbn=R
33,DK000003,사랑스런 코펜하겐!!!!!!!!!!!!!!!!!,경제학과,학부,2011-2012,1,0.947517,/partner/expReport.asp?id=5733&page=4&bgbn=R


In [110]:
# listing up dataframes to concat
list_df = []
for csv_file in abstract_yonsei_reviews:
    # extract file name
    file_name = csv_file.split("/")[-1]
    file_name_without_ext = file_name.split(".")[0]

    # extract university code
    univ_code = file_name_without_ext.split("_")[0]

    df_temp = make_abstract_review_df(univ_code)
    list_df.append(df_temp)

print(len(list_df))
list_df[:3]

470


[       대학코드                제목               학과  과정    년도  BERT_SCORE  \
 0  730       리투아니아에서 행복했던 4개월  사회복지학과/언론홍보영상학부  학부  2019           1   
 
    RNN_SCORE                                           href  
 0   0.752021  /partner/expReport.asp?id=16663&page=1&bgbn=R  ,
        대학코드                    제목    학과  과정      년도  BERT_SCORE  RNN_SCORE  \
 0  AR000004  아르헨티나 교환학생(어학연수프로그램)  건축학과  학부  2013-1           0    0.39967   
 
                                            href  
 0  /partner/expReport.asp?id=6947&page=1&bgbn=R  ,
         대학코드                                        제목        학과  과정  \
 0   AT000001                       쿠프슈타인에서 노후 50년 땡겨쓰기      경영학과  학부   
 1   AT000001                           평화로운 쿠프에서의 한 학기        경제  학부   
 2   AT000001                   소중한 경험, 잊지 못할 기억, 쿠프슈타인   언론홍보영상학  학부   
 3   AT000001                        언제나 마음속의 고향인 쿠프슈타인    중어중문학과  학부   
 4   AT000001                    자연이 아름다운 쿠프슈타인에서의 한 학기    정치외교학과  학부   
 5   AT000001           

In [112]:
# concat dataframe
df_abstract_wrangle = pd.concat(list_df)
print(len(df_abstract_wrangle.index))
df_abstract_wrangle.head(10)

11183


,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,730,리투아니아에서 행복했던 4개월,사회복지학과/언론홍보영상학부,학부,2019,1,0.752021,/partner/expReport.asp?id=16663&page=1&bgbn=R
0,AR000004,아르헨티나 교환학생(어학연수프로그램),건축학과,학부,2013-1,0,0.399670,/partner/expReport.asp?id=6947&page=1&bgbn=R
0,AT000001,쿠프슈타인에서 노후 50년 땡겨쓰기,경영학과,학부,2017-2018,0,0.416185,/partner/expReport.asp?id=15098&page=1&bgbn=R
1,AT000001,평화로운 쿠프에서의 한 학기,경제,학부,2017ㅡ2018,1,0.816743,/partner/expReport.asp?id=14709&page=1&bgbn=R
2,AT000001,"소중한 경험, 잊지 못할 기억, 쿠프슈타인",언론홍보영상학,학부,2017,1,0.854657,/partner/expReport.asp?id=14264&page=1&bgbn=R
3,AT000001,언제나 마음속의 고향인 쿠프슈타인,중어중문학과,학부,2017,1,0.834800,/partner/expReport.asp?id=14181&page=1&bgbn=R
4,AT000001,자연이 아름다운 쿠프슈타인에서의 한 학기,정치외교학과,학부,2017,1,0.870668,/partner/expReport.asp?id=14102&page=1&bgbn=R
5,AT000001,여유로웠던 쿠프슈타인에서의 한 학기,행정학과,학부,2017,1,0.816663,/partner/expReport.asp?id=14094&page=1&bgbn=R
6,AT000001,쿠프슈타인에서의 한 학기,경제학부,학부,2016,1,0.731432,/partner/expReport.asp?id=13866&page=1&bgbn=R
7,AT000001,여유롭고 소중했던 쿠프슈타인에서의 시간,경제학부,학부,2016,1,0.852968,/partner/expReport.asp?id=13822&page=1&bgbn=R


In [114]:
# check for duplicate rows
df_abstract_wrangle = df_abstract_wrangle.drop_duplicates()
print(len(df_abstract_wrangle.index))

11183


In [113]:
# export wrangled df abstract concat dataframe
df_abstract_wrangle.to_csv("./data_wrangled/df_abstract_wrangle.csv",encoding="utf-8", index=False)

## Wrangle University Dataframe

In [37]:
def yield_BERT_SUM(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    dataframe_input = pd.read_csv(file_path, encoding="utf-8")
    bert_sum = dataframe_input["BERT_SCORE"].sum()
    return bert_sum

In [41]:
def yield_RNN_SUM(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    dataframe_input = pd.read_csv(file_path, encoding="utf-8")
    rnn_sum = dataframe_input["RNN_SCORE"].sum()
    return rnn_sum

In [43]:
def yield_no_of_students(UNIV_CODE):
    dataframe_input = make_abstract_review_df(UNIV_CODE)
    no_of_students = len(dataframe_input.index)
    return no_of_students

In [83]:
# University of Copenhagen Satisfaction Level
print(yield_BERT_SUM("DK000003"))
print(yield_RNN_SUM("DK000003"))

65
49.14911192655563


In [84]:
# University of Hongkong Satisfaction Level
print(yield_BERT_SUM("CN000016"))
print(yield_RNN_SUM("CN000016"))

41
43.059615552425385


In [85]:
# read university dataframe
df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")
df_univ.sample(5)

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation
498,498,62,63,Drew University,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000059&bgbn=A,None
96,96,22,23,Wuhan University,CHINA,교환학생,교환대학,/partner/expReport.asp?ucode=CN000017&bgbn=A,Exists
88,88,14,15,Shanghai Jiao Tong University,CHINA,교환학생,교환대학,/partner/expReport.asp?ucode=CN000011&bgbn=A,None
574,574,138,139,Northeastern University,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000302&bgbn=A,None
423,423,30,31,University of Oxford,UNITED KINGDOM,방문학생SAP(정규),교환대학ISEPSAF,/partner/expReport.asp?ucode=GB000015&bgbn=A,None


In [86]:
# make column for university code
series_query = df_univ["href"].str.split("=", expand=True)[1]
series_univ_code = series_query.str.split("&", expand=True)[0]
df_univ["대학코드"] = series_univ_code
df_univ.head()

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation,대학코드
0,0,0,1,Kabul University,AFGHANISTAN,교환학생,교환대학,/partner/expReport.asp?ucode=AF000001&bgbn=A,None,AF000001
1,1,0,1,Universidad Blas Pascal,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000001&bgbn=A,None,AR000001
2,2,1,2,Universidad Catolica de Cordoba,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000002&bgbn=A,None,AR000002
3,3,2,3,Universidad de Palermo,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000003&bgbn=A,None,AR000003
4,4,3,4,Universidad del Salvador,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000004&bgbn=A,Exists,AR000004


In [62]:
# reorder columns, delete unnecessary columns
df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "visitation", "href"]]
df.head(10)

,대학코드,국가,대학명,파견구분,협정형태,visitation,href
0,AF000001,AFGHANISTAN,Kabul University,교환학생,교환대학,None,/partner/expReport.asp?ucode=AF000001&bgbn=A
1,AR000001,ARGENTINA,Universidad Blas Pascal,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000001&bgbn=A
2,AR000002,ARGENTINA,Universidad Catolica de Cordoba,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000002&bgbn=A
3,AR000003,ARGENTINA,Universidad de Palermo,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000003&bgbn=A
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,Exists,/partner/expReport.asp?ucode=AR000004&bgbn=A
5,AR000005,ARGENTINA,University of Buenos Aires,교환학생,교환대학,None,/partner/expReport.asp?ucode=AR000005&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,Exists,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,Exists,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [34]:
# sort out unvisited universities
df_ever_been = df.loc[df['visitation'] == "Exists"]
df_ever_been.head()

,대학코드,국가,대학명,파견구분,협정형태,visitation,href
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,Exists,/partner/expReport.asp?ucode=AR000004&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,Exists,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,Exists,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [68]:
# number of visited universities in Yonsei history
df_ever_been.shape

(470, 7)

In [76]:
# wrangle university dataframe
def make_univ_df():
    # read university dataset csv file
    df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")

    # get university code
    series_query = df_univ["href"].str.split("=", expand=True)[1]
    series_univ_code = series_query.str.split("&", expand=True)[0]
    df_univ["대학코드"] = series_univ_code
    
    # Making column values for BERT_SUM, RNN_SUM, STUDENT_NO
    list_bert_sum = []
    list_rnn_sum = []
    list_student_no = []
    
    # iterrating over rows of university datframe
    for index, row in df_univ.iterrows():

        # if university is not ever visited, input NaN values
        if row["visitation"] == "None":
            list_bert_sum.append(np.NaN)
            list_rnn_sum.append(np.NaN)
            list_student_no.append(np.NaN)

        # if university is visited, input BERT_SCORE, RNN_SCORE's sum and input number of visitations
        if row["visitation"] == "Exists":
            list_bert_sum.append(yield_BERT_SUM(row["대학코드"]))
            list_rnn_sum.append(yield_RNN_SUM(row["대학코드"]))
            list_student_no.append(yield_no_of_students(row["대학코드"]))
    
    # Make Columns
    df_univ["BERT_SUM"] = list_bert_sum
    df_univ["RNN_SUM"] = list_rnn_sum
    df_univ["NUM_VISITS"] = list_student_no
    
    # Select and order columns
    df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "NUM_VISITS", "BERT_SUM", "RNN_SUM", "href"]]
    return df

In [77]:
df_univ_wrangled = make_univ_df()

In [81]:
print(df_univ_wrangled.shape)
df_univ_wrangled.head(10)

(745, 9)


,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href
0,AF000001,AFGHANISTAN,Kabul University,교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=AF000001&bgbn=A
1,AR000001,ARGENTINA,Universidad Blas Pascal,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000001&bgbn=A
2,AR000002,ARGENTINA,Universidad Catolica de Cordoba,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000002&bgbn=A
3,AR000003,ARGENTINA,Universidad de Palermo,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000003&bgbn=A
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,1.0,0.0,0.399670,/partner/expReport.asp?ucode=AR000004&bgbn=A
5,AR000005,ARGENTINA,University of Buenos Aires,교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000005&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,27.0,17.0,17.952732,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,19.0,16.0,13.359809,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,7.0,5.0,4.657787,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,16.0,14.0,9.696670,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [79]:
df_univ_wrangled.columns

Index(['대학코드', '국가', '대학명', '파견구분', '협정형태', 'NUM_VISITS', 'BERT_SUM',
       'RNN_SUM', 'href'],
      dtype='object')

In [115]:
df_univ_wrangled.shape

(745, 9)

In [116]:
# check for duplicate rows
df_univ_wrangled = df_univ_wrangled.drop_duplicates()
df_univ_wrangled.shape

(745, 9)

In [117]:
# export wrangled df abstract concat dataframe
df_univ_wrangled.to_csv("./data_wrangled/df_univ_wrangled.csv",encoding="utf-8", index=False)